Exercise 2

In [1]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/test.nl
chmod u+rx ipopt

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  libgfortran5
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 589 kB of archives.
After this operation, 2,660 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libgfortran5 amd64 8.4.0-1ubuntu1~18.04 [589 kB]
Fetched 589 kB in 1s (821 kB/s)
Selecting previously unselected package libgfortran5:amd64.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../libgfortran5_8.4.0-1ubuntu1~18.04_amd64.deb ...
Unpacking libgfortran5:amd64 (8.4.0-1ubuntu1~18.04) ...
Setting up libgfortran5:amd64 (8.4.0-1ubuntu1~18.04) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



# The above code also fetches an optimization problem in an '.nl' format. 'nl' format is suitable for solvers, not for humans. Lets call ipopt directly (without using any Python) to solve it.

In [2]:
!./ipopt test.nl



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      230

Total number of variables............................:       30
                     variables with only lower bounds:       30
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

In [3]:
!pip install -q pyomo

     |████████████████████████████████| 9.1 MB 4.9 MB/s 
     |████████████████████████████████| 49 kB 4.9 MB/s 


In [4]:
#Importing the required pacakages
from pyomo.environ import *
import numpy as np

In [5]:
#Creating model
model_ex2 = ConcreteModel()

In [6]:
#Variables declaration

In [7]:
model_ex2.xf = Var()
model_ex2.yf = Var()

In [8]:
model_ex2.xl = Var()
model_ex2.yl = Var()

In [9]:
#Objective function
model_ex2.obj = Objective(expr = ((model_ex2.xf-model_ex2.xl)**2 + (model_ex2.yf-model_ex2.yl)**2), sense=minimize)

In [10]:
#Constraints declaration

In [11]:
model_ex2.constr_1 = Constraint(expr=((model_ex2.xl-200)**2 + (model_ex2.yl-200)**2) <= 2500)

In [12]:
model_ex2.constr_2 = Constraint(expr=(model_ex2.xf + 3*model_ex2.yf) <= 0)

In [13]:
model_ex2.constr_3 = Constraint(expr=(model_ex2.yf - model_ex2.xf) <= 0)

In [14]:
model_ex2.constr_4 = Constraint(expr=(model_ex2.xf + model_ex2.yf +200) >= 0)


In [15]:
model_ex2.constr_5 = Constraint(expr=(model_ex2.yf - model_ex2.xf +200) >= 0)

In [16]:
model_ex2.pprint()

4 Var Declarations
    xf : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    xl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    yf : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    yl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : (xf - xl)**2 + (yf - yl)**2

5 Constraint Declarations
    constr_1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                          : Upper  : Active
        None :  -Inf : (xl - 200)**2 + (yl - 200)**

In [17]:
#Model the problem in Pyomo and solve using Ipopt

In [18]:
opt_ipopt = SolverFactory('ipopt')
result_ex2 = opt_ipopt.solve(model_ex2)
print(result_ex2)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.039669036865234375
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [19]:
print('Solver status:', result_ex2.solver.status)
print('Solver termination condition:',result_ex2.solver.termination_condition)
print('Value of Objective function = ', sqrt(model_ex2.obj()))

Solver status: ok
Solver termination condition: optimal
Value of Objective function =  202.98221256032042


Question 3.

In [20]:
#Details of decision variables

In [21]:
print('End points of the farm :')
print('xf :', model_ex2.xf.value)  
print('yf :', model_ex2.yf.value)

End points of the farm :
xf : 120.00000000096625
yf : -39.99999999699526


In [22]:
print('End points of the lake :')
print('xl :', model_ex2.xl.value)
print('yl :', model_ex2.yl.value)

End points of the lake :
xl : 184.18861162009682
yl : 152.56583486031144


Question 4.

In [23]:
model_ex2.obj.deactivate()

In [24]:
#clear() method clears all references to local variables

In [25]:
#Clearing all the reference to end points of farm held by model_ex2

In [26]:
model_ex2.xf.clear()
model_ex2.yf.clear()

In [27]:
#Clearing all the reference to end points of lake held by model_ex2

In [28]:
model_ex2.xl.clear()
model_ex2.yl.clear()

In [29]:
#Decalartion of new varibales for junction 

In [30]:
model_ex2.xj=Var()
model_ex2.yj=Var()

In [31]:
model_ex2.objn=Objective(expr=((model_ex2.xl-model_ex2.xj)**2 + (model_ex2.yl-model_ex2.yj)**2) + ((model_ex2.xj-model_ex2.xf)**2 + (model_ex2.yj-model_ex2.yf)**2) +(((15-model_ex2.xj)**2 + (20-model_ex2.yj)**2))*0.2,sense=minimize)

In [32]:
#Printing the details of model_ex1 that we have created
model_ex2.pprint()

6 Var Declarations
    xf : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False : False :  Reals
    xj : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    xl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False : False :  Reals
    yf : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False : False :  Reals
    yj : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    yl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False : False :  Reals

2 Objective Declarations
    obj : Size=1, Index=None, Active=False


In [33]:
#Model the problem in Pyomo and solve using Ipopt

In [34]:
opt_ipopt = SolverFactory('ipopt')
result_ex2 = opt_ipopt.solve(model_ex2)
print(result_ex2)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.03145718574523926
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [35]:
print('Solver status:', result_ex2.solver.status)
print('Solver termination condition:',result_ex2.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


Question 6.

In [36]:
#Details of decision variables

In [37]:
print('End points of the farm :')
print('xf :', model_ex2.xf.value)  
print('yf :', model_ex2.yf.value)

End points of the farm :
xf : 94.67459978544952
yf : -31.558199925163777


In [38]:
print('End points of the lake :')
print('xl :', model_ex2.xl.value)  
print('yl :', model_ex2.yl.value)

End points of the lake :
xl : 176.51842419226946
yl : 155.85687343895236


In [39]:
print('End points of the junction :')
print('xj :', model_ex2.xj.value)  
print('yj :', model_ex2.yj.value)

End points of the junction :
xj : 124.63319271714498
yj : 58.317578869903905
